In [6]:
from langchain.chat_models import ChatOpenAI


chat = ChatOpenAI()

b = chat.predict("How many planets are there?")

b

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.